In [17]:
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, HttpUrl, Field
import json

# --- Model Definitions ---

# Forward declaration for recursive models if needed (Pydantic v2 handles this better)
class FlightPlace: pass

class ParentPlace(BaseModel):
    """Simplified structure for the 'parent' field within FlightPlace."""
    flightPlaceId: Optional[str] = None
    displayCode: Optional[str] = None
    name: Optional[str] = None
    type: Optional[str] = None
    # Removed 'country' and 'parent' as they don't exist here based on JSON/errors

class FlightPlace(BaseModel):
    """Represents an origin or destination within a segment."""
    flightPlaceId: Optional[str] = None
    displayCode: Optional[str] = None
    parent: Optional[ParentPlace] = None # Use the simplified parent model
    name: Optional[str] = None
    type: Optional[str] = None
    country: Optional[str] = None

class CarrierBase(BaseModel):
    """Base carrier info common to different contexts."""
    id: Optional[int] = None
    name: Optional[str] = None
    alternateId: Optional[str] = None
    logoUrl: Optional[HttpUrl] = None

class SegmentCarrier(CarrierBase):
    """Carrier info as found within segments (includes allianceId, displayCode)."""
    allianceId: Optional[int] = None
    displayCode: Optional[str] = None

class LegCarrier(CarrierBase):
    """Carrier info as found within legs.carriers (might lack allianceId/displayCode)."""
    # Inherits fields from CarrierBase
    # No allianceId or displayCode here based on JSON structure in legs.carriers
    pass


class Segment(BaseModel):
    id: Optional[str] = None
    origin: Optional[FlightPlace] = None
    destination: Optional[FlightPlace] = None
    departure: Optional[str] = None # Keep as string for simplicity, or use datetime
    arrival: Optional[str] = None   # Keep as string for simplicity, or use datetime
    durationInMinutes: Optional[int] = None
    flightNumber: Optional[str] = None
    marketingCarrier: Optional[SegmentCarrier] = None # Use specific carrier model
    operatingCarrier: Optional[SegmentCarrier] = None # Use specific carrier model
    transportType: Optional[str] = None

class LegLocation(BaseModel):
    """Represents an origin or destination at the Leg level."""
    id: Optional[str] = None
    entityId: Optional[str] = None
    name: Optional[str] = None
    displayCode: Optional[str] = None
    city: Optional[str] = None
    country: Optional[str] = None
    isHighlighted: Optional[bool] = None

class LegCarriers(BaseModel):
    """Structure for carriers within a Leg."""
    marketing: Optional[List[LegCarrier]] = None # Use specific carrier model
    operating: Optional[List[LegCarrier]] = None # Use specific carrier model
    operationType: Optional[str] = None

class Leg(BaseModel):
    id: Optional[str] = None
    origin: Optional[LegLocation] = None
    destination: Optional[LegLocation] = None
    durationInMinutes: Optional[int] = None
    stopCount: Optional[int] = None
    isSmallestStops: Optional[bool] = None
    departure: Optional[str] = None # Keep as string for simplicity, or use datetime
    arrival: Optional[str] = None   # Keep as string for simplicity, or use datetime
    timeDeltaInDays: Optional[int] = None
    carriers: Optional[LegCarriers] = None
    segments: Optional[List[Segment]] = None

class Price(BaseModel):
    raw: Optional[float] = None
    formatted: Optional[str] = None
    pricingOptionId: Optional[str] = None # Note: This seems redundant here, it's part of PricingOption

class Eco(BaseModel):
    # Define fields if 'eco' object can have content, otherwise Optional[Any] or Optional[Dict]
    ecoContenderDelta: Optional[float] = None # Example field based on common patterns

class FarePolicy(BaseModel):
    isChangeAllowed: Optional[bool] = None
    isPartiallyChangeable: Optional[bool] = None
    isCancellationAllowed: Optional[bool] = None
    isPartiallyRefundable: Optional[bool] = None

class PriceDetail(BaseModel):
    updateStatus: Optional[str] = None
    amount: Optional[float] = None

class SegmentIdItem(BaseModel):
    price: Optional[PriceDetail] = None
    segmentIds: Optional[List[str]] = None
    bookingProposition: Optional[str] = None
    agentId: Optional[str] = None
    url: Optional[HttpUrl] = None

class PricingOption(BaseModel):
    agentIds: Optional[List[str]] = None
    price: Optional[PriceDetail] = None
    items: Optional[List[SegmentIdItem]] = None
    pricingOptionId: Optional[str] = None
    fareAttributes: Optional[Dict[str, Any]] = Field(default_factory=dict) # Handle empty dict
    agentNames: Optional[List[str]] = None
    agentName: Optional[str] = None

class FlightData(BaseModel):
    price: Optional[Price] = None
    vendor: Optional[str] = None
    firstCarrier: Optional[str] = None
    url: Optional[HttpUrl] = None
    eco: Optional[Eco] = None # Make Eco optional or handle null
    fareAttributes: Optional[Dict[str, Any]] = Field(default_factory=dict) # Handle empty dict
    farePolicy: Optional[FarePolicy] = None
    hasFlexibleOptions: Optional[bool] = None
    id: Optional[str] = None
    isMashUp: Optional[bool] = None
    isProtectedSelfTransfer: Optional[bool] = None
    isSelfTransfer: Optional[bool] = None
    legs: Optional[List[Leg]] = None
    pricingOptions: Optional[List[PricingOption]] = None
    score: Optional[float] = None
    tags: Optional[List[str]] = None

class Root(BaseModel):
    # The input JSON is a list containing one object, which has the 'data' key.
    # To parse the *list* directly, you'd typically parse the inner object type.
    # If the file *always* contains `[{ "data": [...] }]`, this Root model works
    # when parsing the *first element* of the loaded list.
    data: Optional[List[FlightData]] = None

# --- Parsing Logic ---

# Load your JSON (assuming it's saved as 'skyscan.json')
file_path = 'skyscan.json'
try:
    with open(file_path, "r", encoding='utf-8') as f:
        raw_json_list = json.load(f)

    # Check if the loaded data is a list and not empty
    if isinstance(raw_json_list, list) and raw_json_list:
        # Assuming the structure is always [{ "data": [...] }]
        # Parse the first element of the list using the Root model
        parsed_root = Root(**raw_json_list[0])

        # Example access:
        if parsed_root.data:
            print(f"Successfully parsed {len(parsed_root.data)} flight itineraries.")
            first_flight = parsed_root.data[0]

            print("\n--- First Flight Itinerary ---")
            if first_flight.price:
                print("Price:", first_flight.price.formatted)
            print("Vendor:", first_flight.vendor)
            print("Booking URL:", first_flight.url)

            if first_flight.legs:
                first_leg = first_flight.legs[0]
                print("\nFirst Leg:")
                if first_leg.origin:
                    print("  Origin:", first_leg.origin.name, f"({first_leg.origin.displayCode})")
                if first_leg.destination:
                    print("  Destination:", first_leg.destination.name, f"({first_leg.destination.displayCode})")
                print("  Departure:", first_leg.departure)
                print("  Arrival:", first_leg.arrival)
                print("  Duration:", first_leg.durationInMinutes, "minutes")
                if first_leg.carriers and first_leg.carriers.marketing:
                     print("  Marketing Carrier:", first_leg.carriers.marketing[0].name)
                if first_leg.segments and first_leg.segments[0].marketingCarrier:
                    # Note: Accessing segment carrier details might be more reliable
                    seg_carrier = first_leg.segments[0].marketingCarrier
                    print(f"  Segment Marketing Carrier: {seg_carrier.name} ({seg_carrier.displayCode}, Alliance: {seg_carrier.allianceId})")


        else:
            print("Parsed data is empty.")

    else:
        print(f"Error: Expected a non-empty list in {file_path}, but got: {type(raw_json_list)}")

except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from {file_path}")
except Exception as e:
    print(f"An unexpected error occurred during parsing: {e}")

Successfully parsed 12 flight itineraries.

--- First Flight Itinerary ---
Price: 222 €
Vendor: Ryanair
Booking URL: https://www.skyscanner.net/transport_deeplink/4.0/HR/en-GB/EUR/ryan/2/18486.16574.2025-07-07,16574.18486.2025-07-09/air/airli/flights?itinerary=flight|-31915|3104|18486|2025-07-07T09:05|16574|2025-07-07T10:30|145|-|-|-,flight|-31915|2190|16574|2025-07-09T11:30|18486|2025-07-09T14:45|135|-|-|-&carriers=-31915&operators=-30543,-30543&passengers=2;1&channel=website&cabin_class=economy&fps_session_id=03e9e27f-7359-49a2-9ec3-d268d22c4687&ticket_price=221.42&is_npt=false&is_multipart=false&client_id=skyscanner_website&request_id=191df64e-d760-4f2f-9289-347dd128deb3&q_ids=H4sIAAAAAAAA_-NS5mIpqkzME2Lh2DaBUYqFY18jo0LDnDWH2YxYFZi0GBmzmDyCALL3C3AlAAAA|-3224042361253344422|1,H4sIAAAAAAAA_-NS5mIpqkzME2Lh2NfIKMXCsW0Co0LDo22H2YxYFZi0GBmzmDyCAKAkb94lAAAA|3833571988903060114|1&q_sources=JACQUARD,JACQUARD&commercial_filters=false&q_datetime_utc=2025-04-17T19:22:55&pqid=false&fare_type=bas